In [ ]:
#numpy for math, pandas for data manipulation and matplotlib for data visualization.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#for training neural network, pytorch library is used.
import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [ ]:
#if the GPU is available use it for the computation otherwise use the CPU.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#There are 2 ways to load the Fashion MNIST dataset.

#There are 2 ways to load the Fashion MNIST dataset.

#1.   Load csv and then inherite Pytorch Dataset class .
#2.   Use Pytorch module torchvision.datasets. It has many popular datasets like MNIST, FashionMNIST, CIFAR10 e.t.c

#Note: We use DataLoader class from torch.utils.data to load data in batches in both method.

In [ ]:
# this code initializes two variables, train_set and test_set, with the FashionMNIST dataset.
# Using FashionMNIST class from torchvision module.
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))

In [ ]:
# Calculating the length and the answer is 60000
len(train_set)

In [ ]:
# The code you provided sets up data loaders for the FashionMNIST dataset in PyTorch.
#  Data loaders are a convenient way to load and iterate over mini-batches of data during the training and evaluation of a model.
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=100)

In [ ]:
# len is 600
len(train_loader)

In [ ]:
def output_label(label):
    # Mapping of numeric labels to string descriptions
    output_mapping = {
        0: "T-shirt/Top",
        1: "Trouser",
        2: "Pullover",
        3: "Dress",
        4: "Coat",
        5: "Sandal",
        6: "Shirt",
        7: "Sneaker",
        8: "Bag",
        9: "Ankle Boot"
    }
    
    # Check if the label is a tensor, and if so, extract the scalar value
    input = (label.item() if type(label) == torch.Tensor else label)
    
    # Return the corresponding string description from the mapping
    return output_mapping[input]


In [ ]:

#Building a CNN
"""
Make a model class (FashionCNN in our case)
It inherit nn.Module class that is a super class for all the neural networks in Pytorch.
Our Neural Net has following layers:

Two Sequential layers each consists of following layers-

Convolution layer that has kernel size of 3 * 3, padding = 1 (zero_padding) in 1st layer and padding = 0 in second one. Stride of 1 in both layer.
Batch Normalization layer.
Acitvation function: ReLU.
Max Pooling layer with kernel size of 2 * 2 and stride 2.
Flatten out the output for dense layer(a.k.a. fully connected layer).
3 Fully connected layer with different in/out features.
1 Dropout layer that has class probability p = 0.25.
All the functionaltiy is given in forward method that defines the forward pass of CNN.
Our input image is changing in a following way:
First Convulation layer : input: 28 * 28 * 3, output: 28 * 28 * 32
First Max Pooling layer : input: 28 * 28 * 32, output: 14 * 14 * 32
Second Conv layer : input : 14 * 14 * 32, output: 12 * 12 * 64
Second Max Pooling layer : 12 * 12 * 64, output: 6 * 6 * 64
Final fully connected layer has 10 output features for 10 types of clothes.
Lets implementing the network...

"""

In [ ]:
class FashionCNN(nn.Module):
    # Define the FashionCNN class as a subclass of nn.Module
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        # Initialize the FashionCNN class and its parent class (nn.Module)
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Define the first layer of the network as a sequence of operations:
        # - Convolutional layer with 1 input channel, 32 output channels, and a 3x3 kernel with padding of 1
        # - Batch normalization layer for normalization and stabilization
        # - ReLU activation function for introducing non-linearity
        # - Max pooling layer with a kernel size of 2x2 and stride of 2
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Define the second layer of the network as a sequence of operations:
        # - Convolutional layer with 32 input channels, 64 output channels, and a 3x3 kernel
        # - Batch normalization layer
        # - ReLU activation function
        # - Max pooling layer with a kernel size of 2x2
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        # Define the first fully connected (linear) layer with 64*6*6 input features and 600 output features
        
        self.drop = nn.Dropout2d(0.25)
        # Define a dropout layer with a dropout probability of 0.25
        
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        # Define the second fully connected (linear) layer with 600 input features and 120 output features
        
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        # Define the third fully connected (linear) layer with 120 input features and 10 output features
        
    def forward(self, x):
        # Define the forward pass of the network
        
        out = self.layer1(x)
        # Pass the input through the first layer
        
        out = self.layer2(out)
        # Pass the output of the first layer through the second layer
        
        out = out.view(out.size(0), -1)
        # Reshape the output into a 1D tensor by flattening it
        
        out = self.fc1(out)
        # Pass the flattened output through the first fully connected layer
        
        out = self.drop(out)
        # Apply dropout to the output
        
        out = self.fc2(out)
        # Pass the output through the second fully connected layer
        
        out = self.fc3(out)
        # Pass the output through the third fully connected layer
        
        return out
        # Return the final output


In [ ]:
#Making a model of our CNN class
"""
Creating a object(model in the code)
Transfering it into GPU if available.
Defining a Loss function. we're using CrossEntropyLoss() here.
Using Adam algorithm for optimization purpose.
"""

In [ ]:
class FashionCNN(nn.Module):
    # Define the FashionCNN class as a subclass of nn.Module
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        # Initialize the FashionCNN class and its parent class (nn.Module)
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Define the first layer of the network as a sequence of operations:
        # - Convolutional layer with 1 input channel, 32 output channels, and a 3x3 kernel with padding of 1
        # - Batch normalization layer for normalization and stabilization
        # - ReLU activation function for introducing non-linearity
        # - Max pooling layer with a kernel size of 2x2 and stride of 2
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Define the second layer of the network as a sequence of operations:
        # - Convolutional layer with 32 input channels, 64 output channels, and a 3x3 kernel
        # - Batch normalization layer
        # - ReLU activation function
        # - Max pooling layer with a kernel size of 2x2
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        # Define the first fully connected (linear) layer with 64*6*6 input features and 600 output features
        
        self.drop = nn.Dropout2d(0.25)
        # Define a dropout layer with a dropout probability of 0.25
        
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        # Define the second fully connected (linear) layer with 600 input features and 120 output features
        
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        # Define the third fully connected (linear) layer with 120 input features and 10 output features
        
    def forward(self, x):
        # Define the forward pass of the network
        
        out = self.layer1(x)
        # Pass the input through the first layer
        
        out = self.layer2(out)
        # Pass the output of the first layer through the second layer
        
        out = out.view(out.size(0), -1)
        # Reshape the output into a 1D tensor by flattening it
        
        out = self.fc1(out)
        # Pass the flattened output through the first fully connected layer
        
        out = self.drop(out)
        # Apply dropout to the output
        
        out = self.fc2(out)
        # Pass the output through the second fully connected layer
        
        out = self.fc3(out)
        # Pass the output through the third fully connected layer
        
        return out
        # Return the final output


In [ ]:

model = FashionCNN()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

In [ ]:
num_epochs = 5
count = 0
# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Lists for knowing classwise accuracy
predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Transfering images and labels to GPU if available
        images, labels = images.to(device), labels.to(device)

        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)

        # Forward pass
        outputs = model(train)
        loss = error(outputs, labels)

        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()

        # Propagating the error backward
        loss.backward()

        # Optimizing the parameters
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):    # It's the same as "if count % 50 == 0"
            total = 0
            correct = 0

            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)

                test = Variable(images.view(100, 1, 28, 28))

                outputs = model(test)

                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()

                total += len(labels)

            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [ ]:
plt.plot(iteration_list, loss_list)  # Plotting the iteration numbers on the x-axis and loss values on the y-axis
plt.xlabel("No. of Iteration")  # Adding a label for the x-axis
plt.ylabel("Loss")  # Adding a label for the y-axis
plt.title("Iterations vs Loss")  # Adding a title for the plot
plt.show()  # Displaying the plot


In [ ]:
plt.plot(iteration_list, accuracy_list)  # Plotting the iteration numbers on the x-axis and accuracy values on the y-axis
plt.xlabel("No. of Iteration")  # Adding a label for the x-axis
plt.ylabel("Accuracy")  # Adding a label for the y-axis
plt.title("Iterations vs Accuracy")  # Adding a title for the plot
plt.show()  # Displaying the plot


In [ ]:
# Looking the Accuracy in each class of FashionMNIST dataset

# Create lists to store the number of correct predictions for each class and the total number of predictions for each class
class_correct = [0. for _ in range(10)]
total_correct = [0. for _ in range(10)]

# Disable gradient calculation to speed up the computation and reduce memory usage
with torch.no_grad():
    # Iterate over the test_loader to get images and labels
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        test = Variable(images)
        outputs = model(test)
        predicted = torch.max(outputs, 1)[1]
        c = (predicted == labels).squeeze()

        # Iterate over each prediction in the batch
        for i in range(100):
            label = labels[i]
            class_correct[label] += c[i].item()
            total_correct[label] += 1

# Iterate over each class
for i in range(10):
    # Calculate the accuracy for each class and print it
    print("Accuracy of {}: {:.2f}%".format(output_label(i), class_correct[i] * 100 / total_correct[i]))
